#### Reference Code

This is a reference code provided to access wind data across different years

In [ ]:
import requests
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
from math import exp

# 1. Define location, year and Turbine specification
api_key =  # Replace this. API same as for PV modeling
email = # Replace this
full_name = # Replace this
affiliation = 'NYU'
reason = 'research'

lat = ## set accordingly
lon = ## set accordingly
year = 2010

In the following snippet, we have provided reference values for Gamesa, you need to change it for the other two turbines. 

Note that the re_enetry margin and the temperature cutoff are assumed to be the same for all turbines.

In [ ]:
# Turbine specifications

### THESE ARE EXAMPLE VALUES. REPLACE WITH YOURS ###
rated_power_kw = 2500 # [kW] Gamesa G126/2500  
turbine_loss=0.19 # [-] turbine efficiency loss  (19% loss)
cut_out_speed=21 # [m/s]  cut out speed Gamesa G126/2500  
reentry_margin = 5  # m/s Hysteresis effect 
Temperature_cutoff=243.15 # [K] -30 degree C
cut_in_speed = 3.5  # [m/s] typical cut-in speed for many turbines
##############



# 2. Download data
# --- Construct WKT (Well-Known Text) string
wkt = f"POINT({lon} {lat})"

# --- WIND Toolkit API endpoint
url = 'https://developer.nrel.gov/api/wind-toolkit/v2/wind/wtk-download.csv'

# --- Attributes at 100 m
attributes = [
    'windspeed_100m',
    'relativehumidity_2m',
    'temperature_100m',
    'pressure_100m',
]

# --- Request parameters
params = {
    'api_key': api_key,
    'wkt': wkt,
    'names': str(year),
    'interval': '60', #hourly data
    'utc': 'true',
    'leap_day': 'false',
    'email': email,
    'full_name': full_name,
    'affiliation': affiliation,
    'reason': reason,
    'attributes': ','.join(attributes)
}

# --- Make the request
print("Sending request...")
response = requests.get(url, params=params)

if response.status_code != 200:
    print("❌ Error downloading data:")
    print(response.text)
else:
    print("✅ Download successful, saving to CSV...")

    # Save raw CSV file
    filename = f'wind_100m_{lat}_{lon}_{year}.csv'
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(response.text)
    print(f"📁 Saved to {filename}")

Once you have downloaded the code, we can see the values obtained in the CSV file. Note that the values obtained will be used to calculate the partial pressure of water using Tetens equation (refer to lecture notes), and eventually to calculatet he power producede each hour by the turbine.

In [ ]:
# --- 3. Load CSV and preprocess
df = pd.read_csv(filename, skiprows=1)

# Rename columns for convenience
df.columns = [c.strip() for c in df.columns]
df.rename(columns={
    'wind speed at 100m (m/s)': 'v_meas',
    'relative humidity at 2m (%)': 'rh',
    'air temperature at 100m (C)': 'T_C',
    'air pressure at 100m (Pa)': 'p_meas'
}, inplace=True)

# --- 4. Calculate air density (n_meas)
T_K = df['T_C'] + 273.15 # convert Celsius to Kelvin
phi = df['rh'] / 100.0 
p_meas = df['p_meas']

# Tetens equation for vapor pressure
p_water = phi * 610.78 * np.exp((17.27 * df['T_C']) / (df['T_C'] + 237.3))

R_dry = 287.1 # [J/(kg K)]
R_water = 461.5 # [J/(kg K)]

n_meas = (p_meas - p_water) / (R_dry * T_K) + p_water / (R_water * T_K)

# --- 5. Correct wind speed to standard air density
v_meas = df['v_meas']
n_std = 1.225
v_std = v_meas * (n_meas / n_std) ** (1 / 3)

Note that once you have obtained the v_std values, you can start calculating the hourly power produced by each turbine
Keep a note of using the following - 
1. Cut in speed
2. Cut out speed
3. Re entry Margin
4. Temperature Cut off
5. Turbine losses (set at 0.19. i.e the maximum capacity factor SHOULD NOT exceed beyond 0.81)

Consider these factors while calculating the capacity factor of each turbine for each hour.

Here is an example of how you might incorporate the above mentioned factors

In [ ]:
### CREATING HYSTERESIS MASK ###

# First wind speed value
first_v = v_std.iloc[0]

# Initialize state based on first wind speed
if first_v <= cut_out_speed:
    state = 1  # running
else:
    state = 0  # off

states = []
reentry_speed = cut_out_speed - reentry_margin

for v in v_std:
    if state == 1 and v > cut_out_speed:
        state = 0  # shut down
    elif state == 0 and v < reentry_speed:
        state = 1  # restart
    states.append(state)

turbine_state = pd.Series(states, index=v_std.index)

In [ ]:
#### THIS IS A REFERENCE CODE - IMPORT THE POWER CURVE CSV Files, interpolate it (we have used cubic spline)
#### and calculate the power output usingf the v_std calculated previously. This is an example for vernova


from scipy.interpolate import CubicSpline

cf_curve_vernova = pd.read_csv("vernova_1.6_cf_curve.csv")   # 'wind_speed' and 'capacity_factor'

# --- Create cubic spline interpolator ---
spline_vernova = CubicSpline(
    cf_curve_vernova["wind_speed"],
    cf_curve_vernova["capacity_factor"],
    bc_type='natural'  # natural spline = 2nd derivative = 0 at boundaries
)

# --- Read input files ---
hourly_vernova = pd.DataFrame({
    "hour": v_std.index + 1,   # hour 1 to 8760
    "wind_speed": v_std.values
})        # must have 'wind_speed' column


# --- Interpolate CF values ---
hourly_vernova["capacity_factor"] = spline_vernova(hourly_vernova["wind_speed"])

# --- Clip values to [0, 1] (spline may slightly overshoot near edges) ---
hourly_vernova["capacity_factor"] = hourly_vernova["capacity_factor"].clip(lower=0, upper=1)


power_output_hourly_vernova = hourly_vernova["capacity_factor"]
rated_power_vernova = 1.6   # [MW]
power_output_hourly_vernova = power_output_hourly_vernova.where(v_std >= cut_in_speed_vernova, 0)
power_output_hourly_vernova = power_output_hourly_vernova.where(v_std <= cut_out_speed_vernova, 0)
power_output_hourly_vernova = power_output_hourly_vernova.where(T_K >= Temperature_cutoff_vernova, 0)
power_output_hourly_vernova = power_output_hourly_vernova * (1 - turbine_loss_vernova)

power_output_hourly_vernova = power_output_hourly_vernova * turbine_state * rated_power_vernova

print("Power Output Hourly (MW):", power_output_hourly_vernova)